# LangChain: Memory（记忆）

当你与那些语言模型进行交互的时候，他们不会记得你之前和他进行的交流内容，这在我们构建一些应用程序（如聊天机器人）的时候，是一个很大的问题---显得不够智能！

因此，在本节中我们将介绍LangChain 中的 Memory(记忆) 模块，即他是如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力

当使用 LangChain 中的记忆组件时，他可以帮助保存和管理历史聊天消息，以及构建关于特定实体的知识。这些组件可以跨多轮对话存储信息，并允许在对话期间跟踪特定信息和上下文。

LangChain 提供了多种记忆类型，包括：

 - ConversationBufferMemory
 - ConversationBufferWindowMemory
 - Entity Memory
 - Conversation Knowledge Graph Memory
 - ConversationSummaryMemory
 - ConversationSummaryBufferMemory
 - ConversationTokenBufferMemory
 - VectorStore-Backed Memory
   
缓冲区记忆允许保留最近的聊天消息，摘要记忆则提供了对整个对话的摘要。实体记忆则允许在多轮对话中保留有关特定实体的信息。

这些记忆组件都是模块化的，可与其他组件组合使用，从而增强机器人的对话管理能力。Memory(记忆)模块可以通过简单的API调用来访问和更新，允许开发人员更轻松地实现对话历史记录的管理和维护。

此次课程主要介绍其中四种记忆模块，其他模块可查看文档学习。

## 大纲

 - ConversationBufferMemory（对话缓存记忆）
 - ConversationBufferWindowMemory（对话缓存窗口记忆）

在LangChain中，Memory指的是大语言模型（LLM）的短期记忆。为什么是短期记忆？那是因为LLM训练好之后（获得了一些长期记忆），它的参数便不会因为用户的输入而发生改变。当用户与训练好的LLM进行对话时，LLM会暂时记住用户的输入和它已经生成的输出，以便预测之后的输出，而模型输出完毕后，它便会“遗忘”之前用户的输入和它的输出。因此，之前的这些信息只能称作为LLM的短期记忆。

为了延长LLM短期记忆的保留时间，则需要借助一些外部存储方式来进行记忆，以便在用户与LLM对话中，LLM能够尽可能的知道用户与它所进行的历史对话信息。

## ConversationBufferMemory（对话缓存记忆）

这种记忆允许存储消息，然后从变量中提取消息。

### 1.首先，让我们导入相关的包和 Zhipu API 秘钥

#### dotenv模块使用解析：

 - 安装方式：pip install python-dotenv
 - load_dotenv()函数用于加载环境变量，
 - find_dotenv()函数用于寻找并定位.env文件的路径
 - 接下来的代码 _ = load_dotenv(find_dotenv()) ，通过find_dotenv()函数找到.env文件的路径，并将其作为参数传递给load_dotenv()函数。load_dotenv()函数会读取该.env文件，并将其中的环境变量加载到当前的运行环境中

In [1]:
#!pip install python-dotenv

In [2]:
from dotenv import load_dotenv,find_dotenv

In [3]:
_ = load_dotenv(find_dotenv())

In [4]:
from langchain_community.chat_models import ChatZhipuAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [5]:
llm = ChatZhipuAI(temperature=0.5)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose = True
)

### 2.开始对话，第一轮

当我们运行predict时，生成了一些提示，如下所见，他说“以下是人类和AI之间友好的对话，AI健谈“等等，这实际上是LangChain生成的提示，以使系统进行希望和友好的对话，并且必须保存对话，并提示了当前已完成的模型链。

In [6]:
conversation.predict(input = "你好，我的名字叫叶赫那英")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我的名字叫叶赫那英
AI:

> Finished chain.


'你好，叶赫那英！很高兴认识你。今天你想聊些什么呢？我们可以讨论文化、科技、日常生活等任何话题。如果你有任何问题，我也会尽力为你解答。'

### 3.第二轮对话

当我们进行下一轮对话时，他会保留上面的提示

In [7]:
conversation.predict(input = "1+1等于几？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我的名字叫叶赫那英
AI: 你好，叶赫那英！很高兴认识你。今天你想聊些什么呢？我们可以讨论文化、科技、日常生活等任何话题。如果你有任何问题，我也会尽力为你解答。
Human: 1+1等于几？
AI:

> Finished chain.


'1+1等于2。这是一个基础的数学问题。如果你有更复杂的问题，也可以问我哦！'

### 4.第三轮对话

为了验证他是否记忆了前面的对话内容，我们让他回答前面已经说过的内容（我的名字），可以看到他确实输出了正确的名字，因此这个对话链随着往下进行会越来越长

In [8]:
conversation.predict(input = "我的名字是啥?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我的名字叫叶赫那英
AI: 你好，叶赫那英！很高兴认识你。今天你想聊些什么呢？我们可以讨论文化、科技、日常生活等任何话题。如果你有任何问题，我也会尽力为你解答。
Human: 1+1等于几？
AI: 1+1等于2。这是一个基础的数学问题。如果你有更复杂的问题，也可以问我哦！
Human: 我的名字是啥?
AI:

> Finished chain.


'你的名字是叶赫那英。如果你有其他问题或者想聊的话题，请随时告诉我！'

### 5.memory.buffer存储了当前为止所有的对话信息

In [9]:
print(memory.buffer)

Human: 你好，我的名字叫叶赫那英
AI: 你好，叶赫那英！很高兴认识你。今天你想聊些什么呢？我们可以讨论文化、科技、日常生活等任何话题。如果你有任何问题，我也会尽力为你解答。
Human: 1+1等于几？
AI: 1+1等于2。这是一个基础的数学问题。如果你有更复杂的问题，也可以问我哦！
Human: 我的名字是啥?
AI: 你的名字是叶赫那英。如果你有其他问题或者想聊的话题，请随时告诉我！


### 6.也可以通过memory.load_memory_variables({})打印历史消息

这里的花括号实际上是一个空字典，有一些更高级的功能，使用户可以使用更复杂的输入，但我们不会在这个短期课程中讨论它们，所以不要担心为什么这里有一个空的花括号。

In [10]:
memory.load_memory_variables({})

{'history': 'Human: 你好，我的名字叫叶赫那英\nAI: 你好，叶赫那英！很高兴认识你。今天你想聊些什么呢？我们可以讨论文化、科技、日常生活等任何话题。如果你有任何问题，我也会尽力为你解答。\nHuman: 1+1等于几？\nAI: 1+1等于2。这是一个基础的数学问题。如果你有更复杂的问题，也可以问我哦！\nHuman: 我的名字是啥?\nAI: 你的名字是叶赫那英。如果你有其他问题或者想聊的话题，请随时告诉我！'}

### 7.添加指定的输入输出内容到记忆缓存区

In [11]:
memory = ConversationBufferMemory()

继续添加新的内容，对话历史都保存下来在了！

In [12]:
memory.save_context(
    {"input":"你好,我是张小三"},
    {"output":"有什么可以帮助你的吗？"}
)

In [13]:
memory.buffer

'Human: 你好,我是张小三\nAI: 有什么可以帮助你的吗？'

In [14]:
memory.load_memory_variables({})

{'history': 'Human: 你好,我是张小三\nAI: 有什么可以帮助你的吗？'}

当我们在使用大型语言模型进行聊天对话时，大型语言模型本身实际上是无状态的。语言模型本身并不记得到目前为止的历史对话。每次调用API结点都是独立的。

聊天机器人似乎有记忆，只是因为通常有快速的代码可以向LLM提供迄今为止的完整对话以及上下文。因此，Memory可以明确地存储到目前为止的所有术语或对话。这个Memory存储器被用作输入或附加上下文到LLM中，以便它可以生成一个输出，就好像它只有在进行下一轮对话的时候，才知道之前说过什么。

In [15]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose = False
)
conversation.predict(input = "我的名字是啥吗？")

'你的名字是张小三。有什么我可以帮你解答的问题吗？如果有任何需要帮助的事情，请尽管告诉我。'

## ConversationBufferWindowMemory（对话缓存窗口记忆）

随着对话变得越来越长，所需的内存量也变得非常长。将大量的tokens发送到LLM的成本，也会变得更加昂贵,这也就是为什么API的调用费用，通常是基于它需要处理的tokens数量而收费的。

针对以上问题，LangChain也提供了几种方便的memory来保存历史对话。 其中，对话缓存窗口记忆只保留一个窗口大小的对话缓存区窗口记忆。它只使用最近的n次交互。这可以用于保持最近交互的滑动窗口，以便缓冲区不会过大

In [16]:
from langchain.memory import ConversationBufferWindowMemory

### 1.向memory添加两轮对话，并查看记忆变量当前的记录

In [17]:
memory = ConversationBufferWindowMemory(k=1)

In [18]:
memory.save_context(
    {"input":"你好"},
    {"output":"什么事"},
)
memory.save_context(
    {"input":"没事随便问问"},
    {"output":"好的，有事您说话"},
)

In [19]:
memory.load_memory_variables({})

{'history': 'Human: 没事随便问问\nAI: 好的，有事您说话'}

### 2.在看一个例子，发现和上面的结果一样，只保留了一轮对话记忆

In [20]:
llm = ChatZhipuAI(temperature=0.5)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose = False
)

### 3.将对话缓存窗口记忆应用到对话链中

In [21]:
conversation.predict(input = "你好，我的名字叫叶赫那英")

'你好，叶赫那英！很高兴认识你。今天你想聊些什么呢？无论是生活中的琐事，还是你感兴趣的知识点，我都愿意和你分享交流。如果你有任何问题，也可以直接问我，我会尽力为你提供详细的答案。'

In [22]:
conversation.predict(input = "1+1等于几？")

'1+1等于2。这是一个基础的数学问题。如果你有更复杂或者更有趣的问题，也欢迎继续提问！'

注意此处！由于这里用的是一个窗口的记忆，因此只能保存一轮的历史消息，因此AI并不能知道你第一轮对话中提到的名字，他最多只能记住上一轮（第二轮）的对话信息

In [23]:
conversation.predict(input = "还记得我的名字吗?")

'作为人工智能助手，我没有持久的记忆能力来记住用户的个人信息，除非在特定的应用场景中，这些信息被明确地提供并允许存储以提供个性化服务。所以，我的回答会是：\n\nAI: 对不起，我没有记住你的名字。由于隐私保护和数据安全的原因，我无法存储个人信息。如果你愿意告诉我，我可以在这个对话中使用你的名字。'

## ConversationTokenBufferMemory（对话token缓存记忆）

In [25]:
# !pip3 install torch torchvision torchaudio

# 下面这段需要访问 huggingface
# from langchain.memory import ConversationTokenBufferMemory
# llm = ChatZhipuAI(temperature=0.0)
# memory = ConversationTokenBufferMemory(llm=llm,max_token_limit=50)
# memory.save_context({"input":"AI是什么？"},{"output":"机器学习人工智能"})
# memory.save_context({"input":"反向传播是什么？"},{"output":"是导数中的链式传导"})
# memory.save_context({"input":"我是歌手第二期好看吗？"},{"output":"我还没看呢"})

## ConversationSummaryBufferMemory（对话摘要缓存记忆）

In [28]:
# 下面这段需要访问 huggingface

# from langchain.memory import ConversationSummaryBufferMemory
# ctx = "我是歌手第二期没什么意思，只有一首歌好听，如果你没看过可以不用看了。"
# llm = ChatZhipuAI(temperature=0.5)
# memory = ConversationSummaryBufferMemory(llm=llm,max_token_limit=50)
# memory.save_context({"input":"AI是什么？"},{"output":"机器学习人工智能"})
# memory.save_context({"input":"反向传播是什么？"},{"output":"是导数中的链式传导"})
# memory.save_context({"input":"我是歌手第二期好看吗？"},{"output":f"{ctx}"})